<a href="https://colab.research.google.com/github/xuhu357/DataAnalysis/blob/master/ch06_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%A1%9C%EB%94%A9%2C_%EC%A0%80%EC%9E%A5%2C_%ED%8C%8C%EC%9D%BC_%ED%98%95%EC%8B%9D_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터베이스 와 함께 사용하기

대부분 application은 파일에서 데이터를 읽어오지 않는다. (대용량의 데이터에 대해서 비효율적이기 때문에)

SQL에서 데이터를 읽어와서 DataFrame에 저장하는 방법은 꽤 직관적이며, pandas에서는 이 과정을 간결하게 해주는 함수가 몇가지 있다. 


In [0]:
import sqlite3

query = """
CREATE TABLE test
(
a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
);
"""

con = sqlite3.connect(':memory:')

In [0]:
con.execute(query)
con.commit()

데이터를 조금 입력해보자

In [0]:
data = [
    ('Atlanta', 'Georgia', 1.25, 6), 
    ('Tallahassee', 'Florida', 2.6, 3), 
    ('Sacramento', 'California', 1.7, 5)
]

stmt = 'INSERT INTO test VALUES(?, ?, ?, ?)'

con.executemany(stmt, data)
con.commit()

대부분의 파이썬 SQL 드라이버는 테이블에 대해 select 쿼리를 수행하면 tuple list를 반환한다.

In [0]:
cursor = con.execute('select * from test')

In [6]:
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

반환된 tuple list를 직접 pd의 생성자에 전달해도 되지만, 

칼럼 이름을 지정해주면 더 편하다. 


In [7]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [0]:
import pandas as pd


In [9]:
pd.DataFrame(rows, columns=list(zip(*cursor.description))[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


데이터베이스에 쿼리를 보내고 매번 이렇게 하는건 좀 귀찮은 일이다. 

그래서 좀 더 쉽게 접근하는 방법을 소개.

그냥 select 문과 con 객체만 넘기면 됨. 

In [0]:
import pandas.io.sql as sql

In [11]:
pd.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


NoSQL DB인 몽고디비에 데이터를 저장하고 불러오는 것은 각자 알아서 하는걸로.

코드는 아래와 같다. 

In [0]:
import pymongo
import pandas as pd
from pymongo import MongoClient

In [0]:
#point the client at mongo URI
client = MongoClient('localhost', port=27017)

#select database
db = client['database_name']

#select the collection within the database
test = db.test

#convert entire collection to Pandas dataframe
test = pd.DataFrame(list(test.find()))